In [1]:
!pip install lightly -q

In [2]:
#dependencies for file management and data visualisation
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
import re
from tqdm import tqdm
import pandas as pd
import gc
import psutil
from matplotlib.animation import FuncAnimation
import seaborn as sns
from IPython.display import HTML
import cv2

#pretraining dependencies
from lightly.data import LightlyDataset

#pretraining of SIMCLR MODEL
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.loss import NTXentLoss
from lightly.models.modules.heads import SimCLRProjectionHead

#pretraining of BarlowTwins MODEL
from lightly.loss import BarlowTwinsLoss
from lightly.models.modules import BarlowTwinsProjectionHead
from lightly.transforms.byol_transform import (
    BYOLTransform,
    BYOLView1Transform,
    BYOLView2Transform,
)

#pretraining on SWaV MODEL
from lightly.loss import SwaVLoss
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes
from lightly.transforms.swav_transform import SwaVTransform


#fine-tuning the models
import torch
import torch.nn as nn
import torch.nn.functional as F


import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader , random_split
import pytorch_lightning as pl
import torchvision
from torchvision import transforms
from pytorch_lightning import seed_everything

#evaluating the models
from sklearn.metrics import log_loss

seed_value = 50

torch.manual_seed(seed_value)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

seed_everything(42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


42

In [3]:
imagenet_path='/kaggle/input/tinyimagenet200/tiny-imagenet-200/train'
final_loc1='/kaggle/natural'
c=0

#os.makedirs(final_loc, exist_ok=True)
os.makedirs(final_loc1)

for directory in os.listdir(imagenet_path):
    c+=1
    directory_loc=os.path.join(imagenet_path,directory,'images')
    for file in os.listdir(directory_loc):
        
        OLD_PATH=os.path.join(directory_loc,file)
        NEW_PATH=os.path.join(final_loc1,file)
        #os.replace(OLD_PATH,NEW_PATH)
        #print(OLD_PATH, NEW_PATH)
        
        
        shutil.copy(OLD_PATH, NEW_PATH)
        
   
    if(c%20==0):
        print(f'{c} DIRECTORIES TRANSFERED')



20 DIRECTORIES TRANSFERED
40 DIRECTORIES TRANSFERED
60 DIRECTORIES TRANSFERED
80 DIRECTORIES TRANSFERED
100 DIRECTORIES TRANSFERED
120 DIRECTORIES TRANSFERED
140 DIRECTORIES TRANSFERED
160 DIRECTORIES TRANSFERED
180 DIRECTORIES TRANSFERED
200 DIRECTORIES TRANSFERED


In [4]:
counter=0
for i in os.listdir(final_loc1):
    counter+=1
print(c,counter)

200 100000


In [5]:
final_loc2='/kaggle/mri'
c=0

In [6]:
mri1_no='/kaggle/input/brain-mri1/no'
mri1_yes='/kaggle/input/brain-mri1/yes'

os.makedirs(final_loc2,exist_ok=True)
for file in os.listdir(mri1_no):
    c+=1
    
    file_path=os.path.join(mri1_no,file)
    ext=file[-4:]
    if(ext=='jpeg'):
        ext='.jpeg'
    new_filename=str(c)+ext
    final_path=os.path.join(final_loc2,new_filename)
    
    shutil.copy(file_path, final_path)
    
for file in os.listdir(mri1_yes):
    c+=1
    
    file_path=os.path.join(mri1_yes,file)
    ext=file[-4:]
    if(ext=='jpeg'):
        ext='.jpeg'
    new_filename=str(c)+ext
    final_path=os.path.join(final_loc2,new_filename)
    
    shutil.copy(file_path, final_path)
    
    

In [7]:
mri2_test='/kaggle/input/brain-mri2/Testing'
mri2_train='/kaggle/input/brain-mri2/Training'

types=['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']

for i in range(4):
    folder_path=os.path.join(mri2_test,types[i])
    for file in os.listdir(folder_path):
        c+=1
        file_path=os.path.join(folder_path,file)
        new_filename=str(c)+'.jpg'
        final_path=os.path.join(final_loc2,new_filename)
        
        shutil.copy(file_path, final_path)

for i in range(4):
    folder_path=os.path.join(mri2_train,types[i])
    for file in os.listdir(folder_path):
        c+=1
        file_path=os.path.join(folder_path,file)
        new_filename=str(c)+'.jpg'
        final_path=os.path.join(final_loc2,new_filename)
        
        shutil.copy(file_path, final_path)      

In [8]:
mri3_yes='/kaggle/input/brain-mri3/yes'
mri3_no='/kaggle/input/brain-mri3/no'

for file in os.listdir(mri3_yes):
    c+=1
    file_path=os.path.join(mri3_yes,file)
    new_filename=str(c)+'.jpg'
    final_path=os.path.join(final_loc2,new_filename)
    
    shutil.copy(file_path,final_path)
    
for file in os.listdir(mri3_no):
    c+=1
    file_path=os.path.join(mri3_no,file)
    new_filename=str(c)+'.jpg'
    final_path=os.path.join(final_loc2,new_filename)
    
    shutil.copy(file_path,final_path)

In [9]:
counter=0
for i in os.listdir(final_loc2):
    counter+=1
print(c,counter)

6517 6517


**PUTTING UP THE FIRST MODEL NOW**

In [10]:
max_epochs=40

class BarlowTwins(pl.LightningModule):
    def __init__(self,backbone):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = backbone
        self.projection_head = BarlowTwinsProjectionHead(512, 2048, 2048)
        self.criterion = BarlowTwinsLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1) = batch[0]
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(self.parameters(), lr=0.06)
        return optim

In [11]:
BT_Imagenet_transform = BYOLTransform(
    view_1_transform=BYOLView1Transform(input_size=32, gaussian_blur=0.0),
    view_2_transform=BYOLView2Transform(input_size=32, gaussian_blur=0.0),
)

BT_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc1,
    transform=BT_Imagenet_transform,
)

BT_Imagenet_dataloader = torch.utils.data.DataLoader(
    BT_Imagenet_dataset,   
    batch_size=16,         
    shuffle=True,           
    num_workers=4
)
        


In [12]:
torch.cuda.empty_cache()
resnet = torchvision.models.resnet18()
BT_ImagenetPT_model = BarlowTwins(nn.Sequential(*list(resnet.children())[:-1]))
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(BT_ImagenetPT_model, BT_Imagenet_dataloader)

Training: |          | 0/? [00:00<?, ?it/s]

In [13]:
BT_Imagenet_transform = BYOLTransform(
    view_1_transform=BYOLView1Transform(input_size=32, gaussian_blur=0.0),
    view_2_transform=BYOLView2Transform(input_size=32, gaussian_blur=0.0),
)

BT_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc2,
    transform=BT_Imagenet_transform,
)

BT_Imagenet_dataloader = torch.utils.data.DataLoader(
    BT_Imagenet_dataset,   
    batch_size=16,         
    shuffle=True,           
    num_workers=4
)

In [14]:
torch.cuda.empty_cache()
#resnet = torchvision.models.resnet18()
BT_ImagenetPT_model2 = BarlowTwins(BT_ImagenetPT_model.backbone)
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(BT_ImagenetPT_model2, BT_Imagenet_dataloader)

Training: |          | 0/? [00:00<?, ?it/s]

In [15]:
def one_hot(n):
    index={0:0,0.5:1,1:2,2:3}
    #print(index)
    #print(index[n])
    y_new=np.zeros(4)
    y_new[index[n]]=1
    #print(y_new)
    return y_new

In [16]:
c=0
df_new = pd.DataFrame(columns=['IMG', 'CDR'])
def make_dataset(path,cdr):
    
    global c, df_new
    
    for i in os.listdir(path):
        c+=1
        image = cv2.imread(os.path.join(path,i))
        image_np = np.array(image)
        image_tensor = transforms.ToTensor()(image_np)
        padded_image = transforms.CenterCrop((224, 224))(transforms.Pad(28)(image_tensor))
        #print(padded_image.shape)
        '''plt.imshow(padded_image.permute(1, 2, 0))
        print('padded_image',padded_image.shape)'''  
        
        new_entry = {
            'IMG': padded_image,
            'CDR': cdr
        } 
        df_new = pd.concat([df_new, pd.DataFrame([new_entry])], ignore_index=True)
        #print(i)


make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented',1)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented',2)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented',0)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented',0.5)
        
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented',1)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented',2)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented',0)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented',0.5)


In [17]:
print(c,df_new.shape)
print(df_new['IMG'][0].shape)

6400 (6400, 2)
torch.Size([3, 224, 224])


In [18]:
x=np.stack(df_new['IMG'].tolist())
y=df_new['CDR']

y_one_hot=[]
for i in y:
    y_one_hot.append(one_hot(i)) 

In [19]:
x=torch.tensor(x)
y=torch.Tensor(y_one_hot)

dataset=TensorDataset(x,y)

train_size=int(0.8*len(x))
val_size=int(0.1*len(x))
temp_size=2*val_size

#print(train_size,val_size,temp_size)

train_dataset, temp_dataset = random_split(dataset, [train_size, temp_size])
val_dataset,test_dataset=random_split(temp_dataset, [val_size,val_size])

# Define batch size
batch_size = 8

# Create a DataLoader for shuffling and batching
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=1,shuffle=True)

/tmp/ipykernel_47/79332094.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  y=torch.Tensor(y_one_hot)


In [20]:
def train_and_val(backbone):
    
    #training the models first on train_dataloader
    
    models=[]
    for i in range(2):
        models.append(ResnetSingleChannel(backbone,4))
        
   
    
    batch_sizes=[64,32]
    
    for model_no in range(2):
        print(f'TRAINING MODEL {model_no+1}')
        print('--------------------------------------------------------------')
        fine_tune_opt = optim.Adam(models[model_no].parameters(), lr=0.001, weight_decay=0.0001)
        ft_loss_fn= nn.CrossEntropyLoss()
        
        batch_size=batch_sizes[model_no]
        
        train_dataloader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)
        

        num_epochs = 10
        total_steps=train_size//batch_size

        for epoch in range(num_epochs):
            i=0

            for x_batch,y_batch in train_dataloader:
                i+=1
                outputs=models[model_no](torch.Tensor(x_batch))
                
                '''print('out',outputs.shape)
                print('y',y_batch.shape)
                print('out',outputs)
                print('y',y_batch)'''
                
                loss = ft_loss_fn(outputs, y_batch)
                fine_tune_opt.zero_grad()
                loss.backward()
                fine_tune_opt.step()

                if (i+1) % 20 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')
    
    # performing validation on validation data_loader
    best_model=-1
    best_loss=np.inf
    for model_no in range(2):
        
        loss_sum=0
        
        for x_batch,y_batch in val_dataloader:
            outputs=models[model_no](torch.Tensor(x_batch))
            loss = ft_loss_fn(outputs, y_batch)
            loss_sum+=loss.item()
            
        if(loss_sum<best_loss):
            best_model=model_no
            best_loss=loss_sum
    
    return models[best_model]
        
        

In [21]:
def softmax(x):
    return F.softmax(x,dim=-1)

def metrics(model):
    
    test_dataloader=DataLoader(test_dataset,batch_size=1,shuffle=True)
    
    y_pred_M=[]
    y_true_M=[]
    c=0
    logloss_sum=0
    
    for x_batch,y_batch in test_dataloader:
        
        c+=1
        y_pred=model(x_batch)

        '''y_pred_M.append(np.argmax(y_pred.detach().numpy()))
        y_true_M.append(np.argmax(y_batch.detach().numpy()))'''

        softmax_probs = softmax(y_pred)

        '''print(y_pred)
        print(softmax_probs)
        print(y_batch)'''

        logloss = log_loss(y_batch.detach().numpy(), softmax_probs.detach().numpy())

        logloss_sum+=logloss
        #print(logloss) 

    return(logloss_sum/c)

In [22]:
#MAKING A NEW NETWORK WHICH USES THE BACKBONE OF THE PRETRAINED MODEL, AND HAS AN INPUT LAYER TO TAKE IN A SINGLE CHANNEL IMAGE, AND THEN PASS INTO 
#BACKBONE, AND FINALLY OUTPUT LAYER WHICH PERFORMS REGRESSION

class ResnetSingleChannel(nn.Module):
    def __init__(self,backbone,channels):
        super(ResnetSingleChannel,self).__init__()
        self.input=nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1, bias=False)
        self.backbone=backbone
        self.output=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=512, out_features=256),
            nn.Linear(in_features=256, out_features=channels)
        )
        
    def forward(self,x):
        x=self.input(x)
        x=self.backbone(x)
        x=self.output(x)
        
        return x

In [23]:
for layer in BT_ImagenetPT_model.backbone:
    for param in layer.parameters():
        param.requires_grad = True
        
BT_MODEL_final=train_and_val(BT_ImagenetPT_model2.backbone)
BT_M_2=metrics(BT_MODEL_final)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/10], Step [20/80], Loss: 6.5703
Epoch [1/10], Step [40/80], Loss: 4.9681
Epoch [1/10], Step [60/80], Loss: 3.8694
Epoch [1/10], Step [80/80], Loss: 2.2077
Epoch [2/10], Step [20/80], Loss: 2.2728
Epoch [2/10], Step [40/80], Loss: 2.1596
Epoch [2/10], Step [60/80], Loss: 1.9772
Epoch [2/10], Step [80/80], Loss: 1.5075
Epoch [3/10], Step [20/80], Loss: 1.1825
Epoch [3/10], Step [40/80], Loss: 2.1609
Epoch [3/10], Step [60/80], Loss: 1.1725
Epoch [3/10], Step [80/80], Loss: 0.9854
Epoch [4/10], Step [20/80], Loss: 0.8533
Epoch [4/10], Step [40/80], Loss: 0.8921
Epoch [4/10], Step [60/80], Loss: 0.9886
Epoch [4/10], Step [80/80], Loss: 0.9208
Epoch [5/10], Step [20/80], Loss: 1.6053
Epoch [5/10], Step [40/80], Loss: 1.3355
Epoch [5/10], Step [60/80], Loss: 1.7651
Epoch [5/10], Step [80/80], Loss: 2.4242
Epoch [6/10], Step [20/80], Loss: 1.0503
Epoch [6/10], Step [40/80], Loss: 0.7691
Epoch [6/10], Step

In [24]:
print(BT_M_2)

1.0218779558013367
